This notebook is used to create a csv file that includes my point projections in a format similar to fplreview.com. This csv file can then be used for making advanced team optimization plans using Sertalp B. Cay's repo "fpl-optimization".

In [1]:
latest_gameweek = 1

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import requests

pd.set_option('display.max_columns', None)

# Save team data
Info on your own team; starting point for the optimization.

In [3]:
# get login credentials for fetching team data
file_path = Path('../../login_credentials/fpl_login.json')
#file_path = Path('../../login_credentials/fpl_login2.json')
login_credentials = pd.read_json(file_path, typ='series')

In [4]:
# utility function for fetching team data
from src.utils import fetch_my_team

In [5]:
# fetch my team data
my_team = fetch_my_team(login_credentials.user_name, login_credentials.password, login_credentials.team_id)
# drop assistant manager if exists
my_team['picks'] = [item for item in my_team['picks'] if item.get('position') != 16]
my_team

KeyError: 'picks'

In [ ]:
my_team

In [ ]:
# save team data to fpl-optimization repo
file_path = Path('../../../fpl-optimization/data/team.json')
with open(file_path, 'w') as f:
    json.dump(my_team, f)

# Edit projections into fplreview format 
### (to be used with fpl-optimization repo solver)

In [6]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
if 'id' in projections.columns:
    projections = projections.drop('id', axis=1)
display(projections.head())
display(projections.shape)

,name,element_type,home,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,corners_and_indirect_freekicks_order,creativity_rank,direct_freekicks_order,ict_index_rank,influence_rank,minutes,now_cost,penalties_order,points_per_game,selected_by_percent,threat_rank,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,team_name,opponent_team,date,gameweek,expected_points
0,Aaron Wan-Bissaka,2.0,1,1.293087,1.482557,1.664578,1.739111,0.924578,1.031030,1.191148,1.288395,NaN,22.0,NaN,33.0,58.0,90.0,45.0,NaN,1.0,25.1,52.0,1.140252,1.139748,1.197336,1.258066,1.33087,1.399544,1.522491,1.596276,0.0,16.612647,17.978343,3.639634,0.00136,1.66509,0.0,5.63111,0.014329,0.077857,1.290908,89.983802,3.629903,0.0,15.981441,16.424913,3.511579,0.013671,1.595579,0.0,5.487912,0.020778,0.079256,1.4036,89.61498,3.461637,0.0,15.263434,15.085359,3.222477,0.02858,1.631899,0.0,5.24473,0.024141,0.074867,1.496694,88.378925,3.272412,0.0,15.867004,14.431184,2.994092,0.035196,1.648612,0.0,5.210197,0.023099,0.069485,1.524573,87.296029,3.18016,0.0,1755.0,1376.5,274.0,4.0,155.0,0.0,523.0,1.88,5.95,142.96,8329.0,307.063126,0.0,18.963861,14.873934,2.96074,0.043222,1.674871,0.0,5.651339,0.020315,0.064293,1.544771,3.318007,2.12766,West Ham,Chelsea,2025-08-22T19:00:00Z,2,3.156049
1,Aaron Wan-Bissaka,2.0,0,1.157059,1.142377,1.186244,1.208460,1.433377,1.448771,1.395832,1.388979,NaN,22.0,NaN,33.0,58.0,90.0,45.0,NaN,1.0,25.1,52.0,1.140252,1.139748,1.197336,1.258066,1.33087,1.399544,1.522491,1.596276,0.0,16.612647,17.978343,3.639634,0.00136,1.66509,0.0,5.63111,0.014329,0.077857,1.290908,89.983802,3.629903,0.0,15.981441,16.424913,3.511579,0.013671,1.595579,0.0,5.487912,0.020778,0.079256,1.4036,89.61498,3.461637,0.0,15.263434,15.085359,3.222477,0.02858,1.631899,0.0,5.24473,0.024141,0.074867,1.496694,88.378925,3.272412,0.0,15.867004,14.431184,2.994092,0.035196,1.648612,0.0,5.210197,0.023099,0.069485,1.524573

(1820, 113)

In [ ]:
if latest_gameweek>0:
    filepath = Path('../data/fpl_df.csv')
elif latest_gameweek==0:
    filepath = Path('../data/fpl_df_preseason_online.csv')
else:
    print('Check latest_gameweek!')

fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='25-26']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['id', 'name', 'element_type', 'points_per_game', 'total_points',]]
#df['id'] = df.id.astype(int)
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
display(df.head())
display(df.shape)

In [ ]:
# drop duplicate players (some players get new spelling for their name during the season causing duplicates)
duplicate_ids = df.loc[df.id.duplicated(), 'id'].unique()
for id in duplicate_ids:
    ix = df.loc[df.id==id, 'games_played'].idxmin()
    df = df.drop(ix)
display(df.head())
display(df.shape)

In [ ]:
# merge id info to projections
projections = projections.merge(df[['id', 'name']], on='name', how='left')
projections.head()

In [ ]:
# add xMins variable that is needed later
projections['xMins'] = 90

In [ ]:
projections[['id', 'expected_points', 'xMins', 'gameweek']]

In [ ]:
projections_pivot = (
    projections
    .pivot_table(
    columns=['gameweek',],
    index='id', 
    values=['expected_points','xMins'], 
    aggfunc='sum'
    )
)

projections_pivot.head()

In [ ]:
new_cols = []
for col in projections_pivot.columns:
    if col[0] == 'expected_points':
        new_col = str(col[1]) + '_Pts'
        new_cols.append(new_col)
    elif col[0] == 'xMins':
        new_col = str(col[1]) + '_xMins'
        new_cols.append(new_col)

projections_pivot.columns = new_cols
projections_pivot = projections_pivot.reset_index()
projections_pivot = projections_pivot.rename(columns={'id':'ID'})

projections_pivot.head()

In [ ]:
position_dict = {1:'G', 2:'D', 3:'M', 4:'F'}
df['Pos'] = df['element_type'].map(position_dict)
df.rename(columns={'id':'ID'}, inplace=True)    

In [ ]:
projections_pivot = pd.merge(projections_pivot, df[['ID', 'Pos']], on='ID', how='left')
projections_pivot.head()

In [ ]:
# 25-26 pre-season adjustments for defensive contributions

def_contribution_ids = [111, 257, 291, 506,  24,  85,  87, 129, 167, 173, 241, 272, 273,
       302, 457, 459, 517, 519, 585, 586, 587, 643]

cols = ['1_Pts','2_Pts','3_Pts','4_Pts','5_Pts','6_Pts','7_Pts','8_Pts','9_Pts','10_Pts']

display(projections_pivot.loc[projections_pivot.ID.isin(def_contribution_ids),['ID']+cols])

for id in def_contribution_ids:
    projections_pivot.loc[projections_pivot['ID']==id, cols] += 1

display(projections_pivot.loc[projections_pivot.ID.isin(def_contribution_ids),['ID']+cols])

In [ ]:
filepath = Path('../../../fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

### Option to make manual changes, e.g., to xPts (based on injuries)

### adjust points

If model is underestimating e.g. Haaland currently, test how many points more he would need to be getting in order to be drafed to the team in the optimization.

In [ ]:
projections.loc[projections['name'].str.contains('Wood'), 'id'].unique()
#projections.loc[projections['name'].str.contains('Palme'), 'id'].unique()

In [ ]:
player_id = 477
display(projections.loc[projections.id==player_id, 'name'].unique())
display(projections_pivot[projections_pivot.ID==player_id])

In [ ]:
pts_cols = [col for col in projections_pivot.columns if 'Pts' in col]
projections_pivot.loc[projections_pivot.ID==player_id, pts_cols].sum(axis=1) / 10

In [ ]:
gameweeks = np.arange(latest_gameweek+1,latest_gameweek+11)
add_constant = -0.5

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] + add_constant

display(projections_pivot[projections_pivot.ID==player_id])

### Adjust points for unavailable players or players with uncertain availability

In [ ]:
def adjust_points_via_weights(player_id, gameweeks, weights):
    '''Adjust projected player points via gameweek specific weights indicating probability of playing.

    Input:
    player_id (int): Player ID number.
    gameweeks (list): List of gameweek numbers to be adjusted.
    weights (list): Weights for adjusting points for each gameweek given in parameter gameweeks.
    '''
    if len(projections.loc[projections.id==player_id]) > 0:
        player_name = projections.loc[projections.id==player_id, 'name'].unique().item()
        print(f'Name: {player_name}')
        print('Point predictions before adjustment:')
        display(projections_pivot[projections_pivot.ID==player_id])

        # adjust points via probability of playing
        for i in range(len(gameweeks)):
            projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
                weights[i]*projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts']
            
        print('Point predictions after adjustment:')
        display(projections_pivot[projections_pivot.ID==player_id])
    else:
        print(f'No player with ID {player_id} found in projections!')

### Automatic adjustments

In [ ]:
# fetch online data
fpl_online_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_online_df = pd.DataFrame(fpl_online_data['elements'])

In [ ]:
# get play probabilities for players with injuries
play_probabilities = []
for _, row in fpl_online_df.iterrows():
    news = row['news']
    news_splitted = news.split('-')
    if len(news_splitted) > 1:
        first_3_chars = news_splitted[1].strip()[0:3]
        if first_3_chars == '25%':
            play_probabilities.append(0.25)
        elif first_3_chars == '50%':
            play_probabilities.append(0.5)
        elif first_3_chars == '75%':
            play_probabilities.append(0.75)
        else:
            play_probabilities.append(1.0)
    else:
        play_probabilities.append(1.0)
fpl_online_df['play_probability'] = play_probabilities

In [ ]:
# adjust player points based on play probabilities
# only include players with minutes played (which have existing projections)
prob25_ids = fpl_online_df.loc[(fpl_online_df.play_probability==0.25) & (fpl_online_df.minutes>0), 'id'].values
prob50_ids = fpl_online_df.loc[(fpl_online_df.play_probability==0.5) & (fpl_online_df.minutes>0), 'id'].values
prob75_ids = fpl_online_df.loc[(fpl_online_df.play_probability==0.75) & (fpl_online_df.minutes>0), 'id'].values

for player_id in prob25_ids:
    if latest_gameweek<=35:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1, latest_gameweek+2, latest_gameweek+3], weights=[0.25,0.5,0.75])
    elif latest_gameweek==36:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1, latest_gameweek+2], weights=[0.25,0.5])
    elif latest_gameweek==37:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1], weights=[0.25])
    else:
        print('Check latest_gameweek!')

for player_id in prob50_ids:
    if latest_gameweek<=36:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1, latest_gameweek+2], weights=[0.5,0.75])
    elif latest_gameweek==37:
        adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1], weights=[0.5])
    else:
        print('Check latest_gameweek!')

for player_id in prob75_ids:
    adjust_points_via_weights(player_id=player_id, gameweeks=[latest_gameweek+1], weights=[0.75])

In [ ]:
filepath = Path('../../../fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

### Manual adjustments

In [ ]:
# Check player_id
projections.loc[projections['name'].str.contains('Pir'), 'id'].unique()

In [ ]:
player_id = 348
display(projections.loc[projections.id==player_id, 'name'].unique())

In [ ]:
adjust_points_via_weights(player_id=4, gameweeks=[38], weights=[0.75])

In [ ]:
adjust_points_via_weights(player_id=23, gameweeks=[38], weights=[0.5])

In [ ]:
adjust_points_via_weights(player_id=129, gameweeks=[34,35,36], weights=[0,0,0])

In [ ]:
adjust_points_via_weights(player_id=521, gameweeks=[34,35,36,37,38], weights=[0,0,0,0,0])

In [ ]:
adjust_points_via_weights(player_id=477, gameweeks=[34,35,36,37,38], weights=[0.25,0.25,0.25,0.25,0.25])

In [ ]:
adjust_points_via_weights(player_id=554, gameweeks=[34, 35], weights=[0.5, 0.75])

In [ ]:
adjust_points_via_weights(player_id=473, gameweeks=[31,32,33,34,35,36], weights=[0.25,0.25,0.25,0.25,0.25,0.25])

In [ ]:
adjust_points_via_weights(player_id=398, gameweeks=[30,31], weights=[0.5,0.75])

In [ ]:
adjust_points_via_weights(player_id=447, gameweeks=[32], weights=[0.75])

In [ ]:
adjust_points_via_weights(player_id=541, gameweeks=[30,31], weights=[0,0])

In [ ]:
projections.shape

In [ ]:
projections.head()

In [ ]:
# drop cheap goalkeepers
projections = projections[~((projections.now_cost<42) & (projections.element_type==1))].copy()

In [ ]:
filepath = Path('../../../fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

# Other stuff

In [ ]:
# check ids
projections.loc[projections['name'].str.contains('Wissa'), ['id','name']]

In [ ]:
projections.loc[projections.id==218, ['name','team_name']]

In [ ]:
# locked
projections.loc[projections.id.isin([470, 348, 263]), ['id','name']].groupby('id').first()

In [ ]:
# banned players from solver 
projections.loc[projections.id.isin([20, 379, 138, 610, 119, 19, 8, 363, 135, 668, 24, 499, 30, 111, 11]), ['id','name']].groupby('id').first()

In [ ]:
projections[projections.id==498]

In [ ]:
projections_pivot.columns